# **1) Import all packages**

In [ ]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import math
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split 
from numpy.random import seed
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K
from keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
tf.random.set_seed(42)
seed(42)

# **2) Get Dataset**

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/default of credit card clients.xls', index_col=0)

In [ ]:
print(df.head())
print(df.shape)
print(df.shape)

           X1   X2         X3  ...       X22       X23                           Y
ID  LIMIT_BAL  SEX  EDUCATION  ...  PAY_AMT5  PAY_AMT6  default payment next month
1       20000    2          2  ...         0         0                           1
2      120000    2          2  ...         0      2000                           1
3       90000    2          2  ...      1000      5000                           0
4       50000    2          2  ...      1069      1000                           0

[5 rows x 24 columns]
(30001, 24)
(30001, 24)


In [ ]:
print(df.columns)

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23', 'Y'],
      dtype='object')


# **3) Undersample Dataset**

In [ ]:
df['Y'].value_counts()

0                             23364
1                              6636
default payment next month        1
Name: Y, dtype: int64

In [ ]:
 x = df.isna().sum()
 print(x)

X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
X11    0
X12    0
X13    0
X14    0
X15    0
X16    0
X17    0
X18    0
X19    0
X20    0
X21    0
X22    0
X23    0
Y      0
dtype: int64


In [ ]:
# Shuffle the Dataset.
shuffled_df = df.sample(frac=1,random_state=4)

# Put all the fraud class in a separate dataset.
ones_df = shuffled_df.loc[shuffled_df['Y'] == 1]

#Randomly select 492 observations from the non-fraud (majority class)
zeros_df = shuffled_df.loc[shuffled_df['Y'] == 0].sample(n=6636,random_state=42)

# Concatenate both dataframes again
normalized_df = pd.concat([ones_df, zeros_df])

df = normalized_df
df['Y'].value_counts()
labels = df.pop('Y')

In [ ]:
labels_array = labels[1:len(labels)].to_numpy() 
print(labels_array)
labelsv2 = np.vstack(labels_array)
print(labelsv2)
labels[1:len(labels)].value_counts(normalize=True) * 100

[1 1 1 ... 0 0 0]
[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


0    50.003768
1    49.996232
Name: Y, dtype: float64

In [ ]:
#Split data
X_train, X_test, y_train, y_test =  train_test_split(df[1:len(df)], labelsv2, train_size=0.85, test_size=0.15, random_state=42)

In [ ]:
#Scale data
# Training Data
X_train = MinMaxScaler().fit_transform(X_train)
# Testing Data
X_test = MinMaxScaler().fit_transform(X_test)

In [ ]:
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/CC_train.npy",X_train)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/CC_trainlabels.npy",y_train)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/CC_test.npy",X_test)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/CC_testlabels.npy",y_test)

# **4) Vanilla Decision Tree**

In [ ]:
#Create decision tree for specified depth of 5 and 10 
model = DecisionTreeClassifier(max_depth=5, random_state=16)
model.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=16, splitter='best')

In [ ]:
#Get predictions of vanilla DT
res_pred = model.predict(X_test)

In [ ]:
#Get various scores
acc = accuracy_score(y_test, res_pred)
rec = recall_score(y_test, res_pred,average='macro')
prec = precision_score(y_test, res_pred,average='macro')
auc = roc_auc_score(y_test,res_pred, average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.6042189854344551
Recall:  0.6107308379696477
Precision: 0.6480912061794415
AUC:  0.6107308379696477


In [ ]:
#X_train = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/Data/CC_train.npy")
#y_train = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/Data/CC_trainlabels.npy")
#X_test = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/Data/CC_test.npy")
#y_test = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/UCI Breast Cancer Dataset/Data/CC_testlabels.npy")

# **5) Baseline Model**

In [ ]:
log_X_train = X_train
log_y_train = y_train
log_X_test = X_test
log_y_test = y_test

In [ ]:
#Create Logistic Regression Model
logreg = LogisticRegression(max_iter=3000,random_state=16)
logreg.fit(log_X_train,log_y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=16, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#Get AUC Score of Baseline
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
print(logit_roc_auc)

0.6189363571190565


# **6) Built Neural Network**

In [ ]:
print(X_train.shape)
print(X_train)
print(y_train.shape)
X_train = np.asarray(X_train).astype('float32')

(11280, 23)
[[0.15189873 0.         0.5        ... 0.00694236 0.00903614 0.00771004]
 [0.11392405 0.         0.16666667 ... 0.00329993 0.0039759  0.00330359]
 [0.18987342 1.         0.33333333 ... 0.         0.         0.        ]
 ...
 [0.05063291 0.         0.16666667 ... 0.00378358 0.00515663 0.00991326]
 [0.05063291 0.         0.33333333 ... 0.00462824 0.01204819 0.        ]
 [0.16455696 1.         0.16666667 ... 0.00170088 0.00316566 0.00141016]]
(11280, 1)


In [ ]:
print(X_test.shape)
print(y_test.shape)
print(y_test)
X_test = np.asarray(X_test).astype('float32')

(1991, 23)
(1991, 1)
[[1]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]]


In [ ]:
#Prepare Labels
y_train = np.array(keras.utils.to_categorical(y_train))
y_test = np.array(keras.utils.to_categorical(y_test))
print(y_train.shape)
print(y_train[:10])

(11280, 2)
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
#Metrics for Neural Network

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

val_auc = tf.keras.metrics.AUC()

In [ ]:
#Function to built model
def build_model():
  inputs = layers.Input(shape=(23))

  x = layers.Dense(23,activation='relu',name='fc1')(inputs)

  x = layers.Dense(20,activation='relu',name='fc2')(x)
  x = layers.Dense(10,activation='relu',name='fc3')(x)

  x = layers.Dense(6,activation='relu',name='fc4')(x)
  x = layers.Dense(4,activation='relu',name='fc5')(x)

  x = layers.Dense(2,name='logits')(x)

  preds = layers.Activation('softmax',name='Softmax')(x)

  model = Model(inputs=inputs, outputs=preds)
  model.summary()
  return model
model = build_model()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
fc4 (Dense)                  (None, 6)                 66        
_________________________________________________________________
fc5 (Dense)                  (None, 4)                 28        
_________________________________________________________________
logits (Dense)               (None, 2)                

In [ ]:
#Saving best performing model
filepath = "/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Downsampling/Downsampled Models/ModelV15.hdf5"

In [ ]:
#Set-up filepath and ModelCheckepoint with Earlystopping to save best performing moddel based on Validation AUC
model_checkpoint = ModelCheckpoint(filepath,  monitor='val_auc', save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_auc', mode='max', verbose=0, patience=5)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC(name='auc')])

In [ ]:
#Set epochs and bach size, and train model 
num_epoch = 100
batch_size = 64
model1 = model.fit(x=X_train, y=y_train, epochs=num_epoch, batch_size=batch_size, validation_split=0.15,callbacks=[model_checkpoint,es])

Epoch 1/100
150/150 [==============================] - 2s 12ms/step - loss: 0.6609 - acc: 0.6413 - f1_m: 0.6415 - precision_m: 0.6415 - recall_m: 0.6415 - auc: 0.6755 - val_loss: 0.6235 - val_acc: 0.6554 - val_f1_m: 0.6582 - val_precision_m: 0.6582 - val_recall_m: 0.6582 - val_auc: 0.7103
Epoch 2/100
150/150 [==============================] - 0s 3ms/step - loss: 0.6051 - acc: 0.6748 - f1_m: 0.6748 - precision_m: 0.6748 - recall_m: 0.6748 - auc: 0.7340 - val_loss: 0.6018 - val_acc: 0.6826 - val_f1_m: 0.6825 - val_precision_m: 0.6825 - val_recall_m: 0.6825 - val_auc: 0.7354
Epoch 3/100
150/150 [==============================] - 0s 3ms/step - loss: 0.5912 - acc: 0.6869 - f1_m: 0.6870 - precision_m: 0.6870 - recall_m: 0.6870 - auc: 0.7510 - val_loss: 0.6009 - val_acc: 0.6738 - val_f1_m: 0.6731 - val_precision_m: 0.6731 - val_recall_m: 0.6731 - val_auc: 0.7377
Epoch 4/100
150/150 [==============================] - 0s 3ms/step - loss: 0.5828 - acc: 0.6931 - f1_m: 0.6931 - precision_m: 0.6931

In [ ]:
#Train Performance
prediction_model = tf.keras.models.load_model(filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_train, y_train, verbose=0, batch_size=64)

print('Train Accuracy:',accuracy)
print('Train Recall:',recall)
print('Train Precision:',precision)
print('Train F1:',f1_score)
print('Train AUC:',auc)
prediction_model.summary()

Train Accuracy: 0.7119680643081665
Train Recall: 0.7123940587043762
Train Precision: 0.7123940587043762
Train F1: 0.7123939990997314
Train AUC: 0.784024715423584
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
fc4 (Dense)                  (None, 6)                 66        
_________________________________________________________________
fc5 (Dense)             

In [ ]:
#Test Performance
#filepath =  "/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Downsampling/Downsampled Models/ModelV14.hdf5"
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

print('Test Accuracy:',accuracy)
print('Test Recall:',recall)
print('Test Precision:',precision)
print('Test F1:',f1_score)
print('Test AUC:',auc)
prediction_model.summary()

Test Accuracy: 0.625313937664032
Test Recall: 0.6277901530265808
Test Precision: 0.6277901530265808
Test F1: 0.627790093421936
Test AUC: 0.6889231204986572
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
fc4 (Dense)                  (None, 6)                 66        
_________________________________________________________________
fc5 (Dense)                  (

In [ ]:
#Best Performance
best_filepath =  "/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Downsampling/Downsampled Models/ModelV14.hdf5"
best_model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = best_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

#print('Best Accuracy:',accuracy)
#print('Best Recall:',recall)
#print('Best Precision:',precision)
#print('Best F1:',f1_score)
print('Best AUC:',auc)
best_model.summary()

Best AUC: 0.7073178291320801
Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
______________________

# **7) Generate Soft Targets**

In [ ]:
#Load the best model
model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m}) 

In [ ]:
#Compile it again
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC()])
model.summary()

Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
___________________________________________________

In [ ]:
#Remove last softmax layer and predict again
model_sans_softmax = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = model_sans_softmax.predict(X_train)

In [ ]:
#Get the logits
soft_targets = model_logits

In [ ]:
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/CC_Logits.npy", model_logits)

# **8) Knowledge Distillation Decision Tree**


In [ ]:
#Set up Decision Tree for Knowledge Distillation at given depth 
model = DecisionTreeRegressor(max_depth=10, random_state=16)

In [ ]:
#Train Decision Tree
model.fit(X_train,soft_targets)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=16, splitter='best')

In [ ]:
#Get predictions
res_pred = model.predict(X_test)

In [ ]:
# Apply softmax function to output of Decision Tree
pred_argmax = np.zeros_like(res_pred)
pred_argmax[np.arange(len(res_pred)), res_pred.argmax(1)] = 1 

In [ ]:
#Get scores of student decision tree
acc = accuracy_score(pred_argmax,y_test)
rec = recall_score(y_test, pred_argmax,average='macro')
prec = precision_score(y_test, pred_argmax,average='macro')
auc = roc_auc_score(y_test, pred_argmax,average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.676544450025113
Recall:  0.6800912790169331
Precision: 0.69386672579723
AUC:  0.6800912790169331
